# It is just another modified copy of a public notebook

https://www.kaggle.com/code/swimmy/median-model-jpx

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
import matplotlib.pyplot as plt

# Load data

In [ ]:
prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")

In [ ]:
prices.describe()

In [ ]:
pd.DataFrame(prices.isnull().sum())

In [ ]:
NDAYS = 35
lastdays = prices[prices["Date"]>=prices.Date.iat[-2000*NDAYS]].reset_index(drop=True)
ranks = lastdays.groupby("SecuritiesCode")["Target"].mean().rank(ascending=False,method="first").astype(int) -1
pred = ranks.reset_index()
pred = dict(zip(pred["SecuritiesCode"],pred["Target"]))

# Let's take a look at the top 16 best stocks

In [ ]:
plt.figure(figsize=(16, 16), dpi=80)
for i in range(16):
    plt.subplot(4, 4, i+1)
    best_stock_id = pd.DataFrame(ranks).sort_values("Target").index[i]
    plt.plot(prices[prices.SecuritiesCode == best_stock_id].Open)
plt.show()

# Submission

In [ ]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    sample_prediction['Rank'] = sample_prediction["SecuritiesCode"].map(pred)
    env.predict(sample_prediction)